## Power analysis (Monte-Carlo Mapped Power, MCMP)

Goal:
- In this notebook we will reproduce the paper's calculation using MCMP, of the power to detect a drug effect, comparing bivariate vs univariate models.
- Steps:
  1. Simulate a large "reference" population dataset under alternative hypotheses (SLP treatment effects).

  2. Using population parameter values (true or fitted), compute per-subject individual OFV (iOFV) = -2 * log_lik(subject | reduced) + 2 * log_lik(subject | full). (Paper: difference in Objective Functional Value between reduced and full to test for ignoring correlation.)

  3. For each target sample size `n`, repeatedly sample (with replacement) `n` iOFVs from the large population to compute the empirical distribution of sum(iOFV). The proportion of resamples with sum(iOFV) above the chi-square threshold gives power.
  
- Uses Monte Carlo approximation to marginalize over random effects (same approach as in notebook 03_estimation).


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path 

from tqdm.auto import tqdm 
import time, joblib
import matplotlib.pyplot as plt
from scipy.stats import chi2 

import sys
sys.path.insert(0, str(Path('...').resolve()))

from mHMM.src.emissions import EmissionModel
from mHMM.src.transitions import TransitionModel 

from mHMM.utils.sampling_utils import sample_g_matrix_em_params
from mHMM.utils.math_utils import logsumexp_arr
from mHMM.utils.forward_utils import forward_loglik_subject


Check notebook 03_estimation for some of the imported modules

In [ ]:
def subject_loglike_mc(obs, times, init_probs, trans_mat, em_params, K=200,
                       rng_seed=None):
    """MC approximation of the marginal log-likelihood for a single subject.
    Uses: 
    -log-space forward algorithm
    - draws random effects g from variance params (check sample_g_matrix)
    -stable log-mean-exp combination
    
    Arguments are the same as before"""

    em = EmissionModel(**em_params)

    x2_params = {k: em_params[k] for k in ['x2_FEV1R', 'x2_FEV1E', 'x2_PROR', 'x2_PROE']}
    gs = sample_g_matrix_em_params(K, rng_seed, x2_params)

    logls = np.array([
        forward_loglik_subject(obs, times, init_probs, trans_mat, em, g) for g in gs
    ])

    return logsumexp_arr(logls) - np.log(K)

Build Large Population Dataset for Comparison

In [ ]:
POP_N = 500
T_WEEKS = 60
init_probs = np.array([0.9, 0.1])
rng = np.random.default_rng(12345)

#Ref population
pop_em_params_base = dict(
    hFEV1R=3.0, hFEV1E=0.5,
    x2_FEV1R=0.03, x2_FEV1E=0.03,
    hPROR=2.5, hPROE=0.5,
    x2_PROR=0.09, x2_PROE=0.09,
    r2_FEV1=0.015, r2_PRO=0.05,
    qR=-0.33, qE=-0.33,
    PE=0.2, PHL=10.0
)

trans_params_base = {'hpRE':0.1, 'hpER':0.3}

#write a func to simulate single subject given pop params
def simulate_subject_from_pop(em_params, trans_params, rng, T_weeks=60):
    em= EmissionModel(**{k: em_params[k] for k in ['hFEV1R', 'hFEV1E', 'x2_FEV1R', 'x2_FEV1E', 'hPROR', 'hPROE', 'x2_PROR', 'x2_PROE', 'r2_FEV1', 'r2_PRO', 'qR', 'qE', 'PE', 'PHL']})
    tm = TransitionModel(hpRE=trans_params['hpRE'], hpER=trans_params['hpER'])
    trans_mat = tm.transition_matrix
    g = em.sample_individual_effects(rng)
    times = np.arange(T_weeks)
    states = np.zeros(T_weeks, dtype=int)
    states[0]= rng.choice([0,1], p=init_probs)
    obs = np.zeros(T_weeks, 2)

    for t in range(1, T_weeks):
        states[t] = rng.choice([0,1], p=trans_mat[states[t-1], :])
        for t in range(T_weeks):
            mu = np.array([em.inidvidual_fev1(g, states[t]), em.individual_pro(g, t, states[t])])
            cov = em.emission_cov(states, t)
            obs[t] = rng.multivariate_normal(mu, cov, rng)

        return obs, times, states
    
    pop_subjects = []
    for i in tqdm(range(POP_N), desc="Simulate population"):
        obs, times, states = simulate_subject_from_pop(pop_em_params_base, trans_params_base, rng, T_WEEKS)
        pop_subjects.append({'obs': obs, 'times':times, 'states':states})
        


Compute per-subject iOFV for full vs reduced models (drug effect vs w/o)

Usually the model without the treatment slope effect (e.g., SLP=0) or with q=0 for the specific test is the reduced model. 
The paper compared bivariate full vs reduced (uncorrelated) and FEV1-only vs bivariate — we implement a general interface.

In [ ]:
def iOFV_for_subject(subject, init_probs, em_params_full, trans_params_full,
                     em_params_reduced, trans_params_reduced, K=200):
    obs = subject['obs']
    times = subject['times']

    tm_full = TransitionModel(hpRE=trans_params_full['hpRE'], hpER=trans_params_full['hpER'])
    tm_red = TransitionModel(hpRE=trans_params_reduced['hpRE'], hpER=trans_params_reduced['hpER'])
    trans_full = tm_full.transition_matrix()
    trans_red = tm_red.transition_matrix()

    #find marginal log likelihoods
    logl_full = subject_loglike_mc(obs, times, init_probs, trans_full, em_params_full, K=K)
    logl_red = subject_loglike_mc(obs, times, init_probs, trans_red, em_params_reduced, K=K)
    iOFV = -2.0 * (logl_red - logl_full)

    return iOFV 

#for the first 20 subjects:
K = 300 # g samples for marginalization
from joblib import Parallel, delayed
n_jobs = 8

#we define our two model parameter sets as
# (A) Full bivariate vs Reduced (q=0)
em_full = pop_em_params_base.copy()
em_reduced = pop_em_params_base.copy()
em_reduced['qR'] = 0.0
em_reduced['qE'] =  0.0

trans_full = trans_params_base.copy()
trans_reduced = trans_params_base.copy()

#Parallel compute iOFVs for whole population (heavy)
start = time.time()
iOFVs = Parallel(n_jobs=n_jobs)(delayed(iOFV_for_subject)(
    subj, init_probs, em_full, trans_full, em_reduced, trans_reduced, K
) for subj in tqdm(pop_subjects))

end = time.time()
print(f"Computed iOFVs for {len(iOFVs)} subjects in {(end-start)/60:.2f} min")
iOFVs = np.array(iOFVs)

Resampling for power vs sample size

In [ ]:
#degree of freedom equals constrained parameters difference
df_test = 1
chi2_crit = chi2.ppf(0.95, df_test) #our significance criterion is 0.05

def estimate_power_from_iOFVs(iOFVs_population, sample_sizes, n_resamples=1000):
    rng = np.random.default_rng()
    powers = {}
    Npop = len(iOFVs_population)
    for n in sample_sizes:
        exceed = 0
        for _ in range(n_resamples):
            draw = rng.choice(iOFVs_population, size=n, replace=True)
            s =draw.sum() #to find the % sum of iOFVs later and determine power
            if s > chi2_crit:
                exceed +=1
        powers[n] = exceed/ n_resamples 

    return powers 

sample_sizes = [10, 20, 30, 50, 75, 100, 150, 200]
powers = estimate_power_from_iOFVs(iOFVs, sample_sizes, n_resamples=5000, rng_seed=42)
pd.Series(powers)

Plot Power Curve

In [ ]:
plt.figure(figsize=(7,4))
sizes = np.array(list(powers.keys()))
vals = np.array(list(powers.values()))

plt.plot(sizes, vals, marker ='o')
plt.xlabel("Sample size per arm/ group(n)")
plt.ylabel("Power (MCMP)")
plt.title("Power curve (MCMP) - Full vs Reduced")
plt.grid(True)
plt.ylim(0.1, 0.2)
plt.show()

### Repeat for alternative comparisons

FEV1-only model vs bivariate

Change SLP (treatment slope) to 0.5, 1.0, 2.0 and repeat (our paper compared SLP values)

Code on how to set SLP in transition or treatment effect parameter and re-run simulation → compute iOFVs.

In [ ]:
SLPs = [0.5, 1.0, 2.0]
results_by_slp = {}
for slp in SLPs:
    trans_params_alt = {"hpRE":0.1, "hpER":0.3, "gpRE":0.0, "gpER":0.0, "trt":1, "slp":slp}
    #simulate new pop_subjects with slp effect active (treatment effect)
    pop_subs_slp = []
    rng = np.random.default_rng(100 + int(slp*100)) 
    for i in range(POP_N):
        obs, times, states = simulate_subject_from_pop(pop_em_params_base, trans_params_alt, rng, T_WEEKS)
        pop_subs_slp.append({'obs':obs, 'times':times, 'states':states})
    
    #compute iOFVS and build trans_params with/w/o slp
    trans_with = trans_params_alt.copy()
    trans_without = trans_params_alt.copy()
    iOFVS_Slp = Parallel(n_jobs=n_jobs)(delayed(
        iOFV_for_subject)(
            subj, init_probs, em_full, trans_with, em_full, trans_without, K
        ) for subj in tqdm(pop_subs_slp)
    )
    powers_slp = estimate_power_from_iOFVs(np.array(iOFVS_Slp), sample_sizes, n_resamples=5000, rng_seed=1)
    results_by_slp[slp] = powers_slp 

plt.figure(figsize=(8,5))
for slp, pw in results_by_slp.items():
    plt.plot(sample_sizes, [pw[n] for n in sample_sizes], marker ='o', label=f"SLP={slp}")
plt.xlabel("Sample size")
plt.ylabel('Power')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
outdir = Path("../data/results/power_analysis")
outdir.mkdir(parents=True, exist_ok=True)
np.save(outdir / "iOFVs_full_vs_q0.npy", iOFVs)
pd.DataFrame(powers, index=['power']).T.to_csv(outdir / "power_curve_full_vs_q0.csv")

for slp, pw in results_by_slp.items():
    pd.Series(pw).to_csv(outdir / f"power_slp_{slp}.csv")

print(f"Power analysis saved to {outdir}") 

## Summary
1. We have implemented Monte Carlo Mapped Power analysis to estimate model power under different hypotheses (check alt code)

2. We simulated a large ref population and computed per subject **individual Objective Function Values** for full vs reduced models

3. We used **resampling of iOFVs** to estimate empirical power across varying sample sizes

4. Generated **power curves** for key effects such as correlation, SLP treatment effects

5. The results should replicate the paper's power analysis (using Monte Carlo methods)
